# Sistemas de recomendação

## O que são?

* sistemas que calculam e fornecem itens relevantes a um usuário baseado no conhecimento sobre o usuário, conteúdo e sobre as interações do usuário com os itens (tradução livre de uma definição encontrada em [[1]](https://www.manning.com/books/practical-recommender-systems))


## Importância hoje?

* anedotas
    1. youtube
    2. arxiv

## Exemplos:

* recomendações da Amazon após visualização do livro _Foundations of Natural Language Processing_:

![](resources/imgs/amazon_recommender.png)

* recomendações da Netflix para mim:

![](resources/imgs/netflix_recommender.png)

* **Mais??**

## Quando falamos de sistemas de recomendação...

... devemos definir:

* o quê?

* por quê?

* onde/como?

* quão personalizado?

## Efeitos indesejados

* invasão de privacidade

<img src="resources/imgs/target.png" width="500">

Fonte: [Business Insider](https://www.businessinsider.com/the-incredible-story-of-how-target-exposed-a-teen-girls-pregnancy-2012-2)

* espalhamento de desinformação

<img src="resources/imgs/youtube.png" width="300">

Fonte: [Twitter](https://twitter.com/PeterD_Adams/status/1089583479761788928). Leia mais no [NYT](https://www.nytimes.com/2018/03/10/opinion/sunday/youtube-politics-radical.html#click=https://t.co/cGE67FhlsI).

* quebra de confiança

<img src="resources/imgs/taboo.png" width="600">

Fonte: essas são as recomendações geradas pelo Taboola no final da página do [Business Insider](https://www.businessinsider.com/the-incredible-story-of-how-target-exposed-a-teen-girls-pregnancy-2012-2). Leia mais sobre esse tipo de recomendação em [Fortune](http://fortune.com/2016/11/01/outbrain-taboola/).

## Criando um sistema de recomendação

Nas atividades dessa aula, vamos usar o dataset [goodbooks-10k](https://github.com/zygmuntz/goodbooks-10k), um dataset que contém as avaliações (_ratings_) para os 10 mil livros mais populares no site [goodreads](https://www.goodreads.com/).

Nosso objetivo será criar um sistema de recomendação assertivo; mais especificamente, queremos maximizar a leitura dos livros que recomendamos aos usuários.

### Carregando o dataset

Se você não fez o download do dataset ainda, rode a célula abaixo após descomentá-la (remova o `#`).

Para as próximas atividades, é importante carregar dois dos datasets disponíveis:

* `resources/data/ratings.csv`
* `resources/data/books.csv`

In [18]:
#!git clone https://github.com/zygmuntz/goodbooks-10k.git resources/data

Cloning into 'resources/data'...
remote: Enumerating objects: 66, done.
remote: Total 66 (delta 0), reused 0 (delta 0), pack-reused 66
Unpacking objects: 100% (66/66), done.


In [1]:
import warnings

In [2]:
warnings.filterwarnings('ignore')

In [3]:
import pandas as pd

In [4]:
user_ratings_df = pd.read_csv('resources/data/ratings.csv')

In [5]:
user_ratings_df.head(n=1)

,user_id,book_id,rating
0,1,258,5


In [65]:
book_info_df = pd.read_csv('resources/data/books.csv')
book_info_df = book_info_df[['book_id', 'original_title', 'authors', 'original_publication_year']]

In [66]:
book_info_df.head(n=1)

,book_id,original_title,authors,original_publication_year
0,1,The Hunger Games,Suzanne Collins,2008.0


In [6]:
print('Informações sobre o dataset:\n')
print('* Quantidade de avaliações: {:,}'.format(len(user_ratings_df)))
print('* Quantidade de livros com avaliação: {:,}'.format(len(user_ratings_df['book_id'].unique())))
print('* Quantidade de usuários únicos: {: ,}'.format(len(user_ratings_df['user_id'].unique())))

Informações sobre o dataset:

* Quantidade de avaliações: 5,976,479
* Quantidade de livros com avaliação: 10,000
* Quantidade de usuários únicos:  53,424


### Filtro colaborativo

Há dois tipos de filtro colaborativo: o baseado em usuário (`user user`) e o baseado no conteúdo (`item item`).

<img src="resources/imgs/collaborative_filtering.png" width="400">

#### **Atividade**: construção de um filtro colaborativo baseado em usuário

Etapas:

1. normalizar dataset de ratings: para cada `rating`, será subtraída a média de `ratings` do usuário
2. construir uma matriz de usuários (`user_id`) por itens (`book_id`)
3. encontrar os grupos de usuários similares usando o algoritmo [knn](https://scikit-learn.org/stable/modules/neighbors.html)
4. construir uma função que, dado o id de um usuário, traz $n$ recomendações para ele

** Tempo estimado para a atividade: ** 15 minutos

**Etapa 1:** vamos normalizar o dataset `user_ratings_df`: para cada `rating`, será subtraída a média de `ratings` do usuário

In [7]:
mean_ratings_per_user_df = user_ratings_df \
    .groupby('user_id') \
    .agg({'rating': 'mean'}) \
    .reset_index() \
    .rename(columns={'rating': 'mean_rating'})

In [8]:
mean_ratings_per_user_df.head(n=2)

,user_id,mean_rating
0,1,3.589744
1,2,4.415385


In [9]:
user_ratings_df = user_ratings_df.merge(
    mean_ratings_per_user_df, on='user_id', how='inner')

In [10]:
user_ratings_df.head(n=2)

,user_id,book_id,rating,mean_rating
0,1,258,5,3.589744
1,1,268,3,3.589744


In [11]:
user_ratings_df['rating'] = user_ratings_df['rating'] - user_ratings_df['mean_rating']
user_ratings_df.drop('mean_rating', axis=1, inplace=True)

In [12]:
user_ratings_df.head(n=2)

,user_id,book_id,rating
0,1,258,1.410256
1,1,268,-0.589744


* vamos separar parte do nosso dataset como treino e parte dele como teste

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
train_indices, test_indices = train_test_split(range(len(user_ratings_df)))

In [15]:
train_df, test_df = user_ratings_df.loc[train_indices], user_ratings_df.loc[test_indices]

* podemos verificar se a quantidade de usuários no dataset de treino é a mesma do dataset original

In [16]:
len(train_df['user_id'].unique()) == len(user_ratings_df['user_id'].unique())

True

**Etapa 2: ** vamos construir uma matriz de usuários por itens

```python
# naive approach
user_items_matrix = pd.pivot_table(
    user_ratings_df,
    values='rating',
    index=['user_id'],
    columns=['book_id'],
    aggfunc=lambda x: x
).fillna(0).reset_index().values
``` 

<img src="resources/imgs/memory_error.png" width="800">

* para evitar erros de memória, vamos construir uma [matriz esparsa](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.coo_matrix.html) com os dados!

In [17]:
import numpy as np
from scipy.sparse import coo_matrix

In [18]:
unique_user_ids_df = user_ratings_df[['user_id']].drop_duplicates()
unique_user_ids_df['id'] = np.arange(len(unique_user_ids_df))

In [19]:
train_df = train_df.merge(unique_user_ids_df, on='user_id', how='inner')
test_df = test_df.merge(unique_user_ids_df, on='user_id', how='inner')

In [32]:
def generate_user_interactions_matrix(df):
    data = df['rating'].values
    row = df['id'].values
    col = (df['book_id'] - 1).values
    return coo_matrix((data, (row, col)), shape=(max(row)+1, max(col)+1), dtype=np.float64).tocsr()

In [33]:
user_items_matrix = generate_user_interactions_matrix(train_df)

In [34]:
test_user_items_matrix = generate_user_interactions_matrix(test_df)

In [35]:
user_items_matrix, test_user_items_matrix

(<53424x10000 sparse matrix of type '<class 'numpy.float64'>'
 	with 4482359 stored elements in Compressed Sparse Row format>,
 <53424x10000 sparse matrix of type '<class 'numpy.float64'>'
 	with 1494120 stored elements in Compressed Sparse Row format>)

**Etapa 3:** vamos encontrar os grupos de usuários similares usando o algoritmo knn (k-nearest neighbors), que encontra os k usuários mais similares a um determinado usuário

In [24]:
from sklearn.neighbors import NearestNeighbors

In [48]:
k = 50

In [49]:
nn = NearestNeighbors(n_neighbors=k, metric='cosine')

In [50]:
%%time
nn.fit(user_items_matrix)

CPU times: user 38.4 ms, sys: 59.3 ms, total: 97.8 ms
Wall time: 106 ms


NearestNeighbors(algorithm='auto', leaf_size=100, metric='cosine',
         metric_params=None, n_jobs=1, n_neighbors=50, p=2, radius=1.0)

* vamos fazer um teste com o usuário 0

In [31]:
test_df[(test_df['id'] == 0)].head(n=2)

,user_id,book_id,rating,id
518075,1,128,1.410256,0
518076,1,138,-1.589744,0


In [60]:
distances, nearest_ids = nn.kneighbors(test_user_items_matrix[0], n_neighbors=5)

In [61]:
distances

array([[ 0.74372181,  0.74786006,  0.74851963,  0.75978293,  0.76277675]])

In [59]:
nearest_ids

array([[29009, 26220, 24933, 37786, 36757,   153, 27471, 12679, 30171,
          965]])

**Etapa 4:** vamos construir uma função que recebe o id de um usuário e um número $n > 0$ e retorna $n$ recomendações para aquele usuário

In [62]:
unique_user_ids_df = unique_user_ids_df.set_index('user_id')

In [67]:
book_info_df.head()

,book_id,original_title,authors,original_publication_year
0,1,The Hunger Games,Suzanne Collins,2008.0
1,2,Harry Potter and the Philosopher's Stone,"J.K. Rowling, Mary GrandPré",1997.0
2,3,Twilight,Stephenie Meyer,2005.0
3,4,To Kill a Mockingbird,Harper Lee,1960.0
4,5,The Great Gatsby,F. Scott Fitzgerald,1925.0


In [69]:
user_ratings_df['sign'] = user_ratings_df['rating'].apply(lambda rating: 'positive' if rating >=0 else 'negative')

In [94]:
user_id2idx = unique_user_ids_df['id'].to_dict()

In [95]:
idx2user_id = {idx: user_id for user_id, idx in user_id2idx.items()}

In [105]:
def get_unique_user_id(user_id):
    return unique_user_ids_df.loc[user_id]['id']

def get_info_from_user(user_id):
    ratings_from_one_user_df = user_ratings_df[user_ratings_df['user_id'] == user_id]
    ratings_from_one_user_df = ratings_from_one_user_df.merge(book_info_df, on='book_id', how='left')
    return ratings_from_one_user_df.groupby('sign').agg({'original_title': lambda s: s.tolist()})

def get_already_read(user_id):
    return train_df[train_df['user_id'] == user_id]['book_id'].unique().tolist()

def get_similar_users(user_id):
    unique_user_id = get_unique_user_id(user_id)
    distances, nearest_ids = nn.kneighbors(test_user_items_matrix[unique_user_id], n_neighbors=10)
    nearest_user_ids = [idx2user_id[idx] for idx in nearest_ids[0]]
    return user_ratings_df[user_ratings_df['user_id'].isin(nearest_user_ids)] \
        .sort_values(by='rating', ascending=False)
    
def get_recommended_items_from_CF(user_id, n):
    nearest_user_ratings = get_similar_users(user_id)
    nearest_user_ratings = nearest_user_ratings[
        ~nearest_user_ratings['book_id'].isin(get_already_read(user_id))][:n]
    
    recommended_df = pd.DataFrame()
    recommended_df['book_id'] = nearest_user_ratings['book_id'].tolist()
    return recommended_df.merge(book_info_df, on='book_id', how='left')

In [ ]:
test_df.merge()

In [76]:
test_df.head(n=1)

,user_id,book_id,rating,id
0,7005,143,0.47205,4793


In [109]:
get_info_from_user(7005).iloc[1]['original_title'][:10]

['All the King’s Men',
 'The Things They Carried',
 'The Killer Angels',
 'All the Pretty Horses',
 'A Walk in the Woods',
 'The Stand',
 'To Kill a Mockingbird',
 'Three Cups of Tea ',
 'Possession',
 "The Time Traveler's Wife"]

In [107]:
get_recommended_items_from_CF(7005, 5)

,book_id,original_title,authors,original_publication_year
0,83,A Tale of Two Cities,"Charles Dickens, Richard Maxwell, Hablot Knigh...",1859.0
1,3504,The Known World,Edward P. Jones,2003.0
2,130,The Old Man and the Sea,Ernest Hemingway,1952.0
3,2355,Silas Marner: The Weaver of Raveloe,George Eliot,1861.0
4,47,The Book Thief,Markus Zusak,2005.0


## Fatorização de matrizes - construindo modelos com fatores latentes

Também podemos construir um sistema de recomendação usando técnicas de fatorização de matrizes. A ideia de fatorizar matrizes é reduzir a dimensionalidade e, consequentemente, a esparsidade das matrizes de interações, para gerar melhores recomendações para os usuários.

Veja mais sobre redução de dimensionalidade e modelos de fatores latentes nos seguintes vídeos do curso `Mining Massive Datasets`: [vídeo 1](https://www.youtube.com/watch?v=yLdOS6xyM_Q&list=PLLssT5z_DsK9JDLcT8T62VtzwyW9LNepV&index=46) e [vídeo 2](https://www.youtube.com/watch?v=E8aMcwmqsTg&list=PLLssT5z_DsK9JDLcT8T62VtzwyW9LNepV&index=55).

### SVD (Singular Value Decomposition)

In [64]:
from scipy.sparse.linalg import svds

In [69]:
u, s, vt = svds(user_items_matrix, k=100)

In [73]:
u.shape, s.shape, vt.shape

((53424, 100), (100,), (100, 10000))

In [148]:
from scipy.sparse import csr_matrix

In [155]:
vt = csr_matrix(vt)

In [149]:
v = csr_matrix(np.transpose(vt))

In [137]:
user_0 = test_user_items_matrix.getrow(0)

In [138]:
user_0.shape

(1, 10000)

In [150]:
v.shape

(10000, 100)

In [156]:
new_user_0 = np.dot(np.dot(user_0, v), vt)

In [157]:
new_user_0.shape

(1, 10000)

In [159]:
new_df = pd.DataFrame(columns=[f'book_{k}' for k in range(1, 10001)])

In [174]:
u = np.asarray(user_0.todense()).ravel()

In [176]:
new_u = np.asarray(new_user_0.todense()).ravel()

In [177]:
new_df.loc[0] = u
new_df.loc[1] = new_u

In [178]:
new_df.head()

,book_1,book_2,book_3,book_4,book_5,book_6,book_7,book_8,book_9,book_10,...,book_9991,book_9992,book_9993,book_9994,book_9995,book_9996,book_9997,book_9998,book_9999,book_10000
0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,-0.124653,-0.06444,0.257004,-0.003357,-0.026881,0.038335,-0.10294,0.020795,0.490848,0.373307,...,-0.000362,-0.002611,0.006737,0.00443,-0.005341,-0.004257,0.009189,0.002552,0.000313,-0.005129


In [181]:
indices = np.argsort(new_u)

In [182]:
u==0

array([ True,  True,  True, ...,  True,  True,  True], dtype=bool)

In [189]:
indices = set(np.arange(len(new_u))).difference(set(np.nonzero(u)[0]))

In [ ]:
largest_vals = np.argsort(new_u[])

In [179]:
np.where(u == 0)[0]

array([-0.1246529 , -0.06444014,  0.2570037 , ...,  0.00255161,
        0.00031287, -0.00512864])

In [154]:
new_user_0 = np.dot()

<1x100 sparse matrix of type '<class 'numpy.float64'>'
	with 100 stored elements in Compressed Sparse Row format>

In [126]:
a = np.array([[1, 2, 3]])

In [128]:
b = np.array([[3,2],[5,2],[3,3]])

In [141]:
np.dot(a, b).shape

(1, 2)

### NMF (Non-negative Matrix Factorization)

## Bônus: LDA

Acabamos de reproduzir parte da evolução do [sistema de recomendação do NYT](https://open.blogs.nytimes.com/2015/08/11/building-the-next-new-york-times-recommendation-engine/).

## Mais Bônus: ranking e avaliação